In [3]:
import requests
from bs4 import BeautifulSoup

In [4]:
url1 = "https://www.topuniversities.com/university-rankings/world-university-rankings/2018"
url2 = "https://www.timeshighereducation.com/world-university-rankings/2018/world-ranking#!/page/0/length/100/sort_by/rank/sort_order/asc/cols/stats"
json1 = "https://www.topuniversities.com//sites//default//files//qs-rankings-data//357051.txt"
json2 = "https://www.timeshighereducation.com//sites//default//files//the_data_rankings//world_university_rankings_2018_limit0_369a9045a203e176392b9fb8f8c1cb2a.json"
r = requests.get(json1)
json1_text = r.json()
r = requests.get(json2)
json2_text = r.json()

In [5]:
import pandas as pd
import numpy as np
num_of_rank = 200
data1 = pd.DataFrame(data=json1_text["data"][0:200], index=list(range(num_of_rank)))
data2 = pd.DataFrame(data=json2_text["data"][0:200], index=list(range(num_of_rank)))

In [6]:
d1 = data1.copy(deep=True)
d2 = data2.copy(deep=True)
d1.drop(['cc', 'core_id', 'guide', 'logo', 'nid', 'stars'], axis=1, inplace=True)
d2.drop(['aliases', 'member_level', 'nid', 'rank_order', 'record_type', 'scores_citations_rank', 'scores_citations_rank', 'scores_industry_income_rank',
       'scores_international_outlook_rank', 'scores_overall_rank', 'scores_research_rank', 'scores_teaching_rank', 'subjects_offered', 'url'], axis=1, inplace=True)

In [7]:

url_base = "https://www.topuniversities.com"
div1 = '<div class="faculty-main wrapper col-md-4">'
div2 = '<div class="students-main wrapper col-md-4">'
div3 = '<div class="int-students-main wrapper col-md-4">'
class_name = ["faculty-main wrapper col-md-4", "students-main wrapper col-md-4", "int-students-main wrapper col-md-4"]

In [8]:
d1_ = d1.copy(deep=True)
d1_ = d1_.assign(Total_Faculty=0)
d1_ = d1_.assign(International_Faculty=0)
d1_ = d1_.assign(Total_Student=0)
d1_ = d1_.assign(International_Student=0)

for i in d1_.index:
    url_ = d1_.iloc[i]['url']
    r = requests.get(url_base + url_)
    soup = BeautifulSoup(r.text, 'html.parser')
    lst = []
    for ic, c in enumerate(class_name):
        a = soup.findAll("div", { "class" : c})
        if len(a) > 0:
            for j in a[0].findAll("div", { "class" : "number" }):
                lst.append(int(''.join(ch for ch in j.get_text() if ch.isdigit())))
        if ic == 0:
            lst += [np.nan] * (2-len(lst))
        if ic == 1:
            lst += [np.nan] * (3-len(lst))
        if ic == 2:
            lst += [np.nan] * (4-len(lst))                
    d1_.loc[i, ['Total_Faculty', 'International_Faculty', 'Total_Student', 'International_Student']] = lst


In [9]:
QS = d1_.copy()
Times = d2.copy()

![Explanation Image](https://snag.gy/FqTZ6g.jpg)



In [112]:
banned = set(['University', 'of', 'The']) # Banned, useless words

# Extract paranthesis
# Divide words
# Substract banned words
# to list

QS_words = QS['title'].apply(lambda x: [x.replace(')', '').replace('(','') for x in x.split(' ')]).apply(lambda x: list(set(x) - banned)).tolist()
Times_words = Times['name'].apply(lambda x: [x.replace(')', '').replace('(','') for x in x.split(' ')]).apply(lambda x: list(set(x) - banned)).tolist()

In [126]:
# points are from the same words
points = [[len(set(qs_word_list) & set(times_word_list)) for qs_word_list in QS_words] for times_word_list in Times_words]

In [127]:
times_ids = [i for i in range(200)] # suppose every names from Times are different
qs_ids = [i for i in range(200,400)] # suppose every names from QS are differetent

used_names = []

points = [[len(set(qs_word_list) & set(times_word_list)) for qs_word_list in QS_words] for times_word_list in Times_words]
for idx, point in enumerate(points):
    indices = [i for i, x in enumerate(point) if x == max(point)] # Find maximum points
    for index in indices:
        if index not in used_names:
            if max(point) > 0:
                used_names.append(index)
                qs_ids[index] = idx
            break
        
Times['nameIndex'] = times_ids
QS['nameIndex'] = qs_ids



In [128]:
results = pd.merge(QS, Times, how='outer', on=['nameIndex'])

In [129]:
# Unsuccessfull pairs

for result in results[['name','title']].itertuples():
    if str(result.name) == 'nan' or str(result.title) == 'nan':
        print(result.name, "-", result.title)

nan - The University of Sydney
nan - Tokyo Institute of Technology
nan - Osaka University
nan - Ludwig-Maximilians-Universität München
nan - Universidad de Buenos Aires (UBA)
nan - Tohoku University
nan - Korea University
nan - Yonsei University
nan - Universiti Malaya (UM)
nan - Nagoya University
nan - Universidade de São Paulo
nan - Universidad Nacional Autónoma de México  (UNAM)
nan - Hokkaido University
nan - Kyushu University
nan - Chalmers University of Technology
nan - Pontificia Universidad Católica de Chile (UC)
nan - The Hebrew University of Jerusalem
nan - University of Otago
nan - University of Waterloo
nan - Hanyang University
nan - University of Bath
nan - National Tsing Hua University
nan - University of Bergen
nan - University College Dublin
nan - Politecnico di Milano
nan - Indian Institute of Technology Delhi (IITD)
nan - King Fahd University of Petroleum & Minerals
nan - University of Technology Sydney
nan - CentraleSupélec
nan - Indian Institute of Technology Bombay

In [130]:
# All pairs
results[['title', 'name']]

,title,name
0,Massachusetts Institute of Technology (MIT),Massachusetts Institute of Technology
1,Stanford University,Stanford University
2,Harvard University,Harvard University
3,California Institute of Technology (Caltech),California Institute of Technology
4,University of Cambridge,University of Cambridge
5,University of Oxford,University of Oxford
6,UCL (University College London),University College London
7,Imperial College London,Imperial College London
8,University of Chicago,University of Chicago
9,ETH Zurich - Swiss Federal Institute of Techno...,ETH Zurich – Swiss Federal Institute of Techno...
